# Building Firecrown Cluster Counts Likelihood

In this notebook, we are going to provide a example of how to build the cluster counts likelihood using firecrown to be used in two different manners:

- A likelihood to be used with CosmoSIS

- A likelihood to be used outside CosmoSIS

In [1]:
"""Likelihood factory function for cluster number counts."""

import os
from typing import Any, Dict

import pyccl as ccl

from firecrown.likelihood.gauss_family.statistic.cluster_number_counts import (
    ClusterNumberCounts,
)
from firecrown.likelihood.gauss_family.gaussian import ConstGaussian
from firecrown.modeling_tools import ModelingTools
from firecrown.models.cluster_abundance import ClusterAbundance
from firecrown.models.cluster_mass_rich_proxy import ClusterMassRich
from firecrown.models.cluster_redshift_spec import ClusterRedshiftSpec
from firecrown.sacc_support import sacc


from firecrown.likelihood.likelihood import NamedParameters


## Likelihood function

When used with CosmoSIS, there are some parameters that do not need to be set in the likelihood function, since the .ini files required for CosmoSIS will set these parameters. We need to create a function named `build_likelihood`, which will replace its correspondent abstract function from the Firecrown likelihood class. This function must have a `build_parameters` as input, which is a `NamedParameters` object from the likelihood file inside firecrown. This object must be instantiated with a dictionary containing the parameters for the statiscs objects. 

In [2]:
def build_likelihood(build_parameters):
    """
    Here we instantiate the number density (or mass function) object.
    """
# First block: We will instantiate all the objects that will be passed to ClusterNumberCounts the statistics 
    pivot_mass = 14.625862906
    pivot_redshift = 0.6

    cluster_mass_r = ClusterMassRich(pivot_mass, pivot_redshift)
    cluster_z = ClusterRedshiftSpec()

    hmd_200 = ccl.halos.MassDef200m()
    hmf_args: Dict[str, Any] = {}
    hmf_name = "Tinker08"
    cluster_abundance = ClusterAbundance(hmd_200, hmf_name, hmf_args)
    
################ IMPORTANT ################ set_parameters block
#When using the likelihood in CosmoSIS, this block is not needed. As said, the .ini files will set these parameters for the sampler.
#If we want to use this function in another sampler or just run the likelihood, we need to set the parameters needed for the cluster objects.
#We are setting the parameters for random values to be able to run the likelihood in the end. 

    cluster_mass_r.__setattr__("mu_p0", 3.0)
    cluster_mass_r.__setattr__("mu_p1", .86)
    cluster_mass_r.__setattr__("mu_p2", .0)
    cluster_mass_r.__setattr__("sigma_p0", 3.0)
    cluster_mass_r.__setattr__("sigma_p1", .7)
    cluster_mass_r.__setattr__("sigma_p2", 0.0)
    
 ########### END OF PARAMETERS SET BLOCK ##############


#Second block: Here we will generate the statiscs with the objects above and the parameters passed in the beggining of the function.
    stats = ClusterNumberCounts(
        "numcosmo_simulated_redshift_richness",
        cluster_abundance,
        cluster_mass_r,
        cluster_z,
        use_cluster_counts=build_parameters.get_bool("use_cluster_counts", True),
        use_mean_log_mass=build_parameters.get_bool("use_mean_log_mass", False),
    )
    stats_list = [stats]

    # Here we instantiate the actual likelihood. The statistics argument carry
    # the order of the data/theory vector.

#Third Block: we will create a likelihood (in this case a constant gaussian) with the statiscs object
    lk = ConstGaussian(stats_list)

    # We load the correct SACC file.
    saccfile = os.path.expanduser(
        os.path.expandvars(
            "${FIRECROWN_DIR}/examples/cluster_number_counts/"
            "cluster_redshift_richness_sacc_data.fits"
        )
    )
    sacc_data = sacc.Sacc.load_fits(saccfile)

    # The read likelihood method is called passing the loaded SACC file, the
    # cluster number count functions will receive the appropriated sections
    # of the SACC file.
    lk.read(sacc_data)

    # This script will be loaded by the appropriated connector. The framework
    # then looks for the `likelihood` variable to find the instance that will
    # be used to compute the likelihood.
    modeling = ModelingTools()

    return lk, modeling


### Running the likelihood
After defining the `build_likelihood` function, it will be called by CosmoSIS for each sampler interation. Here we will call the function ouselves to test our implementation.
Therefore we had to set the parameters values in the likelihood function. If used with CosmoSIS, the `set_parameters` block is not needed. 

In [3]:
build_parameters = NamedParameters({"use_cluster_counts": True,
"use_mean_log_mass": False
})

firecrown_mod = ModelingTools()

cosmo = ccl.Cosmology(
    Omega_c=0.22, Omega_b=0.0448, h=0.71, sigma8=0.8, n_s=0.963, Neff=3.04
)
firecrown_mod.prepare(cosmo)
lk = build_likelihood(build_parameters)
loglike = lk[0].compute_loglike(firecrown_mod)

print(f"The value of -2logL is {loglike}")

The value of -2logL is -10802.132488515948


We can thus use this likelihood function with other samplers. To simplify it, we can change the `build_parameters` to a simple dictionary and make the required changes in the `build_likelihood` function.

If used with CosmoSIS, we don't need to set the parameters in the likelihood build function. At each step, the sampler will set the parameters and run the function in a similar manner to the cell above, giving the likelihood value as an output for the next iteration.